# Machine Learning 2024-2025 - UMONS

# Classification

In this lab we will experiment with multi-class classification. We will consider several models.
We will be using the [Wine Quality](https://archive.ics.uci.edu/ml/datasets/wine+quality) dataset, which contains several attributes of white wines.
Each observation is associated to a rating between $0$ and $10$ that will be the label of our classification task.

The columns of the dataframe contain the following information:
* `fixed_acidity`: amount of tartaric acid in $\text{g}/\text{dm}^3$,
* `volatile_acidity`: amount of acetic acid in $\text{g}/\text{dm}^3$,
* `citric_acid`: amount of citric acid in $\text{g}/\text{dm}^3$,
* `residual_sugar`: amount of remaining sugar after fermentation stops in $\text{g}/\text{l}$,
* `chlorides`: amount of salt in wine,
* `free_sulfur_dioxide`: amount of free $\text{SO}_2$,
* `total_sulfur_dioxide`: amount of free and bound forms of $\text{SO}_2$,
* `density`: density of the wine,
* `pH`: pH level of the wine on a scale from $0$ to $14$,
* `sulphates`: amount of sulphates,
* `alcohol`: the percent of alcohol content,
* `quality`: quality of the wine (score between $0$ and $10$).

**Import the necessary libraries.**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score,
                             confusion_matrix, log_loss, classification_report)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import warnings

np.random.seed(0)

**We load the dataset `wine.csv`.**

In [ ]:
df = pd.read_csv('data/wine.csv', sep=';')
df.head()

**1) Check the properties of this dataset (length, types, missing values).** 

In [ ]:
print(df.shape)
print(df.dtypes)
print(df.isna().sum())

## Data splitting

**We predict the target `quality` from all other features. We split the dataset into a training and test set following a $80/20$ partition.**

In [ ]:
ylabel = 'quality'
X = df.drop(ylabel, axis=1)
y = df[ylabel]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, test_size=0.2, shuffle=True, random_state=0
)

## Data exploration

**2) Look at the distribution of the variable `quality` in the training set using `sns.countplot`.**

In [ ]:
fig = plt.figure(figsize=(4, 2.5))
sns.countplot(x=y_train)

**3) For each continuous feature, we plot a boxplot of this feature grouped by label values. Use the `sns.boxenplot` function of the `seaborn` library. Which features seem to be the most useful to predict the label `quality`?**

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(10, 8), sharex=True)
axes = axes.flatten()
for column, axis in zip(X_train.columns, axes):
    sns.boxenplot(x=y_train, y=X_train[column], ax=axis)
axes[-1].set_visible(False)
fig.tight_layout()

Based on the boxplots, alcohol, density, and pH seem to be the most useful features.

**4) Plot the pairwise relationship of the most useful features using the function `sns.pairplot`. Plot a different color according to the value of the variable `quality` using the `hue` parameter. What do you observe?**

In [ ]:
sns.pairplot(pd.concat([X_train[['alcohol', 'density', 'pH']], y_train], axis=1), hue='quality');

We notice that there is a negative correlation between the variables alcohol and density. There does not seem to be an obvious correlation between the other variables.

## Classification metrics

**5) Define a simple pipeline where you first scale the data with `StandardScaler` to have zero mean and unit variance followed by a (linear) logistic regression. Then, fit the model.**

In [ ]:
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

**6) One of the most useful tools to diagnose a classification model is the confusion matrix. Print it using `confusion_matrix` and `ConfusionMatrixDisplay`.**

The size of the matrix is $n \times n$, where $n$ is the number of classes. Each row represents the instances in an actual class, while each column represents the instances in a predicted class. A cell $i, j$ represents the number of instances of class $i$ that were predicted as class $j$.

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(ax=ax);

**7) From the confusion matrix, several performance metrics can be calculated for each class, as well as overall metrics. Using the function `classification_report`, generate a report of these different metrics. Use the argument `zero_division=0` to avoid warnings.**

Here is what each of these terms represents:

1. **Precision** (also known as *positive predicted value*): This is the ratio of correctly predicted positive observations to the total predicted positives. It is an indicator of the accuracy of the positive predictions. For class $i$, precision is calculated as
   $$
   \text{Precision}_i = \frac{\text{TP}_i}{\text{TP}_i + \text{FP}_i},
   $$
   where $\text{TP}_i$ are the true positives for class $i$ and $\text{FP}_i$ are the false positives for class $i$.

2. **Recall** (also known as *sensitivity* or *true positive rate*): This is the ratio of correctly predicted positive observations to all observations in the actual class. It shows how well the model can find all the positive samples. For class $i$, recall is calculated as
   $$
   \text{Recall}_i = \frac{\text{TP}_i}{\text{TP}_i + \text{FN}_i},
   $$
   where $\text{FN}_i$ are the false negatives for class $i$.

3. **$F_1$ Score**: This is the harmonic mean of precision and recall. Therefore, this score takes both false positives and false negatives into account. It is particularly useful when the class distribution is uneven. $F_1$ score is calculated as
   $$
   \text{$F_1$ Score}_i = \frac{2}{\frac{1}{\text{Precision}_i} + \frac{1}{\text{Recall}_i}} = 2 \times \frac{\text{Precision}_i \times \text{Recall}_i}{\text{Precision}_i + \text{Recall}_i},
   $$
   and is between $0$ and $1$ ($1$ means that all predictions are correct, and $0$ means that there is no correct prediction).

4. **Support**: This is the number of actual occurrences of the class in the specified dataset. It does not reflect the model's performance but is very useful for determining the significance of the classification metrics.

These metrics can be averaged to obtain:
- **Macro average**: This is the average of the precision, recall, and $F_1$ score without taking class imbalance into account. It treats all classes equally, regardless of their support.
- **Weighted average**: This averages the precision, recall, and $F_1$ score, weighted by the support for each class. This means that the influence of each class's score on the overall average is proportional to the number of instances of that class.

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_, zero_division=0))

### Cross-entropy

The **cross-entropy loss** (also called *log loss*, which is the name used in scikit-learn) for a multi-class classification model is calculated as follows:
$$
\text{Cross-Entropy} = -\frac{1}{n} \sum_{i=1}^{n} \sum_{k=1}^{K} y_{ik} \log(p_{ik})
$$
where:
- $n$ is the total number of observations,
- $K$ is the number of classes,
- $y_{ik}$ is a binary indicator: $1$ if class label $k$ is the correct classification for observation $i$, and $0$ otherwise,
- $p_{ik}$ is the predicted probability that observation $i$ belongs to class $k$.

Remember from the course that $\argmin_{\theta \in \Theta} \mathbb{E}[-\log p(Y; \theta)] = \argmin_{\theta \in \Theta} \text{KL}(p_\theta, p)$.
Since the distribution that minimizes the KL divergence is the true distribution, the expectation of the cross-entropy will be minimized when the model always predicts the correct vector of probabilities.

**8) Predict probabilities using the `predict_proba` method of the logistic regression model. Then calculate the cross-entropy using the `log_loss` function.**

In [ ]:
y_pred_proba = model.predict_proba(X_test)
log_loss(y_test, y_pred_proba, labels=model.classes_)

**9) Based on `y_test_binarized`, compute the cross-entropy manually and check that it corresponds to the previous log loss.**

In [ ]:
lb = LabelBinarizer()
lb.fit(y_train)
y_test_binarized = lb.transform(y_test)
y_test_binarized.shape, y_test_binarized[:5]

In [ ]:
-(y_test_binarized * np.log(y_pred_proba)).sum(axis=1).mean()

## Hyperparameter tuning

**10) We will now experiment with various models for classification. For each one of the following models, we design a grid of hyperparameters based on the corresponding scikit-learn documentation:**
- **[KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)**
- **[Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)**
- **[Linear Discriminant Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)**
- **[Quadratic Discriminant Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html)**
- **[Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)** **(see also [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html))**
- **[Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)**
- **[Gradient Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)**

**Make sure you understand all parameters; we will discuss random forests and gradient boosting later.**

In [ ]:
# K-nearest neighbors
param_grid_knn = {
    'clf__n_neighbors': [3, 5, 10, 20, 50, 100], # K
    'clf__weights': ['uniform', 'distance'], # Whether to weigh the neighbors equally or by their distance, as in locally weighted regression
    'clf__p': [1, 2], # Norm 1 or norm 2, i.e., Manhattan or Euclidean distance
}

# Parametric naive Bayes assuming Gaussian distribution of the features.
# This is a very simple model, so we don't need to tune any hyperparameters.
param_grid_nb = {}

# Linear discriminant analysis (LDA)
param_grid_lda = {}

# Quadratic discriminant analysis (QDA)
# The parameter here is a regularization parameter that can be used to reduce overfitting, given that there are many parameters to learn.
param_grid_qda = {
    'clf__reg_param': [0, 0.1, 0.5, 1],
} 

# In scikit-learn, logistic regression regularizes by default, so we need to specify the penalty and the regularization strength.
param_grid_lr = {
    'clf__penalty': ['l2'],
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__fit_intercept': [True, False],
}

# We additionally experiment with a logistic regression model with feature selection
param_grid_lr_skb = {
    'skb__k': [3, 5, 7, 9],
    'clf__penalty': ['l2'],
    'clf__fit_intercept': [True, False],
}

# Random forest
param_grid_rf = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'n_estimators': [100, 200, 300], # Number of trees in the forest
    'max_depth': [None, 2, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Gradient boosting
param_grid_gb = {
    'loss': ['log_loss'],
    'learning_rate': [0.02, 0.1, 0.5],
    'n_estimators': [100, 200, 300], # Number of boosting stages
    'criterion': ['friedman_mse', 'squared_error'], # Function to measure the quality of a split
    'max_depth': [None, 2, 5, 10],
}

## Model fitting

**11) For each one of these models, select the hyperparameters that give the lowest cross-entropy using the [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) class. We start by normalizing the data. Compute the accuracy and cross-entropy on the test dataset for the best hyperparameters (and store them in array).**

**Print the best hyperparameters corresponding to each model and plot a confusion matrix.**

In [ ]:
knn = KNeighborsClassifier()
nb = GaussianNB()
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()

preprocessor = StandardScaler()
knn = Pipeline([('pre', preprocessor), ('clf', knn)])
nb = Pipeline([('pre', preprocessor), ('clf', nb)])
lda = Pipeline([('pre', preprocessor), ('clf', lda)])
qda = Pipeline([('pre', preprocessor), ('clf', qda)])
lr = Pipeline([('pre', preprocessor), ('clf', lr)])
lr_skb = Pipeline([('pre', preprocessor), ('skb', SelectKBest()), ('clf', LogisticRegression(max_iter=1000))])

default_grid_params = dict(n_iter=10, cv=5, n_jobs=4, random_state=0)

grids = {
    'KNN': RandomizedSearchCV(knn, param_grid_knn, scoring='neg_log_loss', **default_grid_params),
    'Naive Bayes': RandomizedSearchCV(nb, param_grid_nb, scoring='neg_log_loss', **default_grid_params),
    'Linear Discriminant Analysis': RandomizedSearchCV(lda, param_grid_lda, scoring='neg_log_loss', **default_grid_params),
    'Quadratic Discriminant Analysis': RandomizedSearchCV(lda, param_grid_lda, scoring='neg_log_loss', **default_grid_params),
    'Logistic Regression': RandomizedSearchCV(lr, param_grid_lr, scoring='neg_log_loss', **default_grid_params),
    'Logistic Regression with feature selection': RandomizedSearchCV(lr_skb, param_grid_lr_skb, scoring='neg_log_loss', **default_grid_params),
    'Random Forest': RandomizedSearchCV(rf, param_grid_rf, scoring='neg_log_loss', **default_grid_params),
    'Gradient Boosting': RandomizedSearchCV(gb, param_grid_gb, scoring='neg_log_loss', **default_grid_params),
}

In [ ]:
results = []
for model_name, model in grids.items():
    print('Running', model_name)
    # Note that by default the argument `refit` of `RandomizedSearchCV` is set to True, so that the best estimator 
    # is refit on the whole training set.
    model.fit(X_train, y_train)

    # We measure the test accuracy and log score.
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {test_accuracy:.3f}')

    y_pred_proba = model.predict_proba(X_test)
    test_log_loss = log_loss(y_test, y_pred_proba, labels=model.classes_)
    print(f'Log loss: {test_log_loss:.3f}')

    results.append([test_accuracy, test_log_loss])
    
    print(f'Best hyperparameters: {model.best_params_}')

    # We plot the confusion matrix
    fig, ax = plt.subplots(figsize=(4, 4))
    cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot(ax=ax)
    plt.show()

**12) Create a pandas dataframe where each row corresponds to a model. The columns should correspond to the accuracy and log loss.**

In [ ]:
pd.DataFrame(results, columns=['Test accuracy', 'Test cross-entropy loss'], index=grids.keys())

We observe that Random Forest, Gradient Boosting, and KNN obtain the best accuracy.
Random Forest and Gradient Boosting obtain the best cross-entropy value.
Note that, on this dataset, KNN has a good accuracy, but its probabilistic predictions are quite poor, as indicated by the log loss.